# Advanced Tutorial 2: Pipeline

## Overview

In this tutorial, we will discuss the following topics:

* [Iterating Through Pipeline](#ta02itp)
    * [Basic Concept](#ta02bc)
    * [Example](#ta02example)
* [Dropping Last Batch](#ta02dlb)
* [Padding Batch Data](#ta02pbd)
* [Benchmark Pipeline Speed](#ta02bps)

In the [Beginner Tutorial 4](../beginner/t04_pipeline.ipynb), we learned how to build a data pipeline that handles data loading and preprocessing tasks efficiently. Now that you have understood some basic operations in the `Pipeline`, we will demonstrate some advanced concepts and how to leverage them to create efficient `Pipelines` in this tutorial.

<a id='ta02itp'></a>

## Iterating Through Pipeline

In many deep learning tasks, the parameters for preprocessing tasks are precomputed by looping through the dataset. For example, in the `ImageNet` dataset, people usually use a precomputed global pixel average for each channel to normalize the images. 

<a id='ta02bc'></a>

### Basic Concept

In this section, we will see how to iterate through the pipeline in FastEstimator. First we will create a sample NumpyDataset from the data dictionary and load it into a `Pipeline`:

In [1]:
import numpy as np
from fastestimator.dataset.data import cifair10
    
# sample numpy array to later create datasets from them
x_train, y_train = (np.random.sample((10, 2)), np.random.sample((10, 1)))
train_data = {"x": x_train, "y": y_train}

In [2]:
import fastestimator as fe
from fastestimator.dataset.numpy_dataset import NumpyDataset

# create NumpyDataset from the sample data
dataset_fe = NumpyDataset(train_data)

pipeline_fe = fe.Pipeline(train_data=dataset_fe, batch_size=3)

Let's get the loader object for the `Pipeline`, then iterate through the loader with a for loop:

In [3]:
with pipeline_fe(mode="train") as loader_fe:
    for batch in loader_fe:
        print(batch)

{'x': tensor([[0.7535, 0.8959],
        [0.3395, 0.3684],
        [0.7611, 0.8407]], dtype=torch.float64), 'y': tensor([[0.3363],
        [0.7955],
        [0.6686]], dtype=torch.float64)}
{'x': tensor([[0.2981, 0.9539],
        [0.7815, 0.0650],
        [0.5846, 0.9597]], dtype=torch.float64), 'y': tensor([[0.8646],
        [0.9627],
        [0.6552]], dtype=torch.float64)}
{'x': tensor([[0.8666, 0.7815],
        [0.9720, 0.1288],
        [0.1624, 0.9450]], dtype=torch.float64), 'y': tensor([[0.9582],
        [0.9136],
        [0.9767]], dtype=torch.float64)}
{'x': tensor([[0.9984, 0.6867]], dtype=torch.float64), 'y': tensor([[0.6442]], dtype=torch.float64)}


<a id='ta02example'></a>

### Example

Let's say we have the ciFAIR-10 dataset and we want to find global average pixel value over three channels:

In [4]:
from fastestimator.dataset.data import cifair10

cifair_train, _ = cifair10.load_data()

We will take the `batch_size` 64 and load the data into `Pipeline`

In [5]:
pipeline_cifair = fe.Pipeline(train_data=cifair_train, batch_size=64)

Now we will iterate through batch data and compute the mean pixel values for all three channels of the dataset. 

In [6]:
with pipeline_cifair(mode="train", shuffle=False) as loader_fe:
    mean_arr = np.zeros((3))
    for i, batch in enumerate(loader_fe):
        mean_arr = mean_arr + np.mean(batch["x"].numpy(), axis=(0, 1, 2))
    mean_arr = mean_arr / (i+1)

In [7]:
print("Mean pixel value over the channels are: ", mean_arr)

Mean pixel value over the channels are:  [125.32287898 122.96682199 113.8856495 ]


<a id='ta02dlb'></a>

## Dropping Last Batch

If the total number of dataset elements is not divisible by the `batch_size`, by default, the last batch will have less data than other batches.  To drop the last batch we can set `drop_last` to `True`. Therefore, if the last batch is incomplete it will be dropped.

In [8]:
pipeline_fe = fe.Pipeline(train_data=dataset_fe, batch_size=3, drop_last=True)

<a id='ta02pbd'></a>

## Padding Batch Data

There might be scenario where the input tensors have different dimensions within a batch. For example, in Natural Language Processing, we have input strings with different lengths. For that we need to pad the data to the maximum length within the batch.


To further illustrate in code, we will take numpy array that contains different shapes of array elements and load it into the `Pipeline`.

In [9]:
# define numpy arrays with different shapes
elem1 = np.array([4, 5])
elem2 = np.array([1, 2, 6])
elem3 = np.array([3])

# create train dataset
x_train = np.array([elem1, elem2, elem3], dtype=object)
train_data = {"x": x_train}
dataset_fe = NumpyDataset(train_data)

We will set any `pad_value` that we want to append at the end of the tensor data. `pad_value` can be either `int` or `float`:

In [10]:
pipeline_fe = fe.Pipeline(train_data=dataset_fe, batch_size=3, pad_value=0)

Now let's print the batch data after padding:

In [11]:
with pipeline_fe(mode="train", shuffle=False) as loader_fe:
    for elem in loader_fe:
        print(elem)

{'x': tensor([[4, 5, 0],
        [1, 2, 6],
        [3, 0, 0]])}


<a id='ta02bps'></a>

## Benchmark Pipeline Speed

It is often the case that the bottleneck of deep learning training is the data pipeline. As a result, the GPU may be underutilized. FastEstimator provides a method to check the speed of a `Pipeline` in order to help diagnose any potential problems. The way to benchmark `Pipeline` speed in FastEstimator is very simple: call `Pipeline.benchmark`.

For illustration, we will create a `Pipeline` for the ciFAIR-10 dataset with list of Numpy operators that expand dimensions, apply `Minmax` and finally `Rotate` the input images: 

In [12]:
from fastestimator.op.numpyop.univariate import Minmax, ExpandDims
from fastestimator.op.numpyop.multivariate import Rotate

pipeline = fe.Pipeline(train_data=cifair_train,
                       ops=[Minmax(inputs="x", outputs="x_out"),
                            Rotate(image_in="x_out", image_out="x_out", limit=180),
                            ExpandDims(inputs="x_out", outputs="x_out", mode="train")],
                       batch_size=64)

Let's benchmark the pre-processing speed for this pipeline in training mode:

In [13]:
pipeline.benchmark(mode="train")

FastEstimator: Step: 100, Epoch: 1, Steps/sec: 391.5610956125886
FastEstimator: Step: 200, Epoch: 1, Steps/sec: 747.2569378198104
FastEstimator: Step: 300, Epoch: 1, Steps/sec: 748.5045903295395
FastEstimator: Step: 400, Epoch: 1, Steps/sec: 723.6569267453751
FastEstimator: Step: 500, Epoch: 1, Steps/sec: 739.2736444296826
FastEstimator: Step: 600, Epoch: 1, Steps/sec: 817.6766785538406
FastEstimator: Step: 700, Epoch: 1, Steps/sec: 810.7956409723804

Breakdown of time taken by Pipeline Operations (train epoch 1)
Op         : Inputs : Outputs :  Time
--------------------------------------
Minmax     : x      : x_out   : 36.47%
Rotate     : x_out  : x_out   : 52.17%
ExpandDims : x_out  : x_out   : 11.35%
